In [ ]:
from pathlib import Path
import pandas as pd

# Low Dimension

In [ ]:
def best_modeling_graph(path_results_validation, dict_, high_dimension):

  basepath = Path(path_results_validation)
  datasets = basepath.iterdir()

  for file_ in datasets:
    if file_.is_file():

      file_name = file_.name
      parts = file_name.split('_')
      dataset = parts[0]
      i=1
      if dataset == 'relevant':
        dataset = 'relevant_reviews'
        i=i+1
      k = parts[i]
      method = parts[i+1]
      if method == 'features':
        method = parts[i+2]
        if not high_dimension:
          method = method + '_' + parts[i+3]

        df_1 = pd.read_csv(path_results_validation + dataset + '_k=1_features_' + method + '_OCSVM.csv', sep=';')
        df_2 = pd.read_csv(path_results_validation + dataset + '_k=2_features_' + method + '_OCSVM.csv', sep=';')
        df_3 = pd.read_csv(path_results_validation + dataset + '_k=3_features_' + method + '_OCSVM.csv', sep=';')

      elif method == 'OLGA':
        if high_dimension:
          df_1 = pd.read_csv(path_results_validation + dataset + '_k=1_' + method + '_128.csv', sep=';')
          df_2 = pd.read_csv(path_results_validation + dataset + '_k=2_' + method + '_128.csv', sep=';')
          df_3 = pd.read_csv(path_results_validation + dataset + '_k=3_' + method + '_128.csv', sep=';')
        else:
          df_1 = pd.read_csv(path_results_validation + dataset + '_k=1_' + method, sep=';')
          df_2 = pd.read_csv(path_results_validation + dataset + '_k=2_' + method, sep=';')
          df_3 = pd.read_csv(path_results_validation + dataset + '_k=3_' + method, sep=';')
          method = method.replace('.csv', '')
      else:
        df_1 = pd.read_csv(path_results_validation + dataset + '_k=1_' + method, sep=';')
        df_2 = pd.read_csv(path_results_validation + dataset + '_k=2_' + method, sep=';')
        df_3 = pd.read_csv(path_results_validation + dataset + '_k=3_' + method, sep=';')
        method = method.replace('.csv', '')


      best_f1_1 = max(df_1['macro avg_f1-score-mean'])
      best_f1_2 = max(df_2['macro avg_f1-score-mean'])
      best_f1_3 = max(df_3['macro avg_f1-score-mean'])

      if best_f1_1 > best_f1_2 and best_f1_1 > best_f1_3:
        dict_[method][dataset] = 'k=1'
      elif best_f1_2 > best_f1_1 and best_f1_2 > best_f1_3:
        dict_[method][dataset] = 'k=2'
      else:
        dict_[method][dataset] = 'k=3'

  return dict_

In [ ]:
path_results_validation = '/content/drive/MyDrive/USP/Doctorate/Research/Articles/OLGA: One cLass Graph Autoencoder/results/8_datasets_2,3_emb/'

dict_ = {'gae_2': {},
            'deepwalk_2': {},
            'node2vec_2': {},
            'gae_3': {},
            'deepwalk_3': {},
            'node2vec_3': {},
            'OC-GCN': {},
            'OC-GAT': {},
            'OC-GraphSAGE': {},
            'OLGA': {}}

dict_validation = best_modeling_graph(path_results_validation, dict_, False)
best_modeling = pd.DataFrame(dict_validation)
best_modeling

,gae_2,deepwalk_2,node2vec_2,gae_3,deepwalk_3,node2vec_3,OC-GCN,OC-GAT,OC-GraphSAGE,OLGA
fakenews,k=1,k=3,k=3,k=3,k=1,k=3,k=2,k=3,k=2,k=3
terrorism,k=3,k=1,k=2,k=3,k=1,k=3,k=3,k=2,k=2,k=2
relevant_reviews,k=2,k=1,k=3,k=3,k=1,k=2,k=3,k=3,k=3,k=2
food,k=2,k=1,k=3,k=2,k=1,k=3,k=3,k=3,k=3,k=2
pneumonia,k=3,k=2,k=3,k=3,k=1,k=3,k=3,k=3,k=3,k=3
musk,k=2,k=1,k=2,k=1,k=1,k=3,k=3,k=3,k=2,k=3
TUANDROMD,k=3,k=3,k=1,k=1,k=3,k=1,k=1,k=1,k=2,k=3
strawberry,k=1,k=1,k=2,k=3,k=1,k=3,k=2,k=3,k=3,k=1


In [ ]:
def results_dataframe(path_results_test, best_modeling):

  dict_test = {'gae': {},
               'gae-std': {},
              'deepwalk': {},
               'deepwalk-std': {},
              'node2vec': {},
               'node2vec-std': {},
              'OC-GCN': {},
                'OC-GCN-std': {},
              'OC-GAT': {},
               'OC-GAT-std': {},
              'OC-GraphSAGE': {},
               'OC-GraphSAGE-std': {},
              'OLGA': {},
              'OLGA-std': {}}

  for dataset in best_modeling.index:
    for method in best_modeling.columns:
      if len(method.split('_')) == 2:
        met = method.split('_')[0]

        df_2 = pd.read_csv(path_results_validation + dataset + '_' + best_modeling[met + '_2'][dataset] + '_features_' + met + '_2_OCSVM.csv', sep=';')
        df_3 = pd.read_csv(path_results_validation + dataset + '_' + best_modeling[met + '_3'][dataset] + '_features_' + met + '_3_OCSVM.csv', sep=';')

        best_f1_2 = max(df_2['macro avg_f1-score-mean'])
        best_f1_3 = max(df_3['macro avg_f1-score-mean'])

        if best_f1_2 > best_f1_3:
          df = pd.read_csv(path_results_test + dataset + '_' + best_modeling[met + '_2'][dataset] + '_features_' + met + '_2_OCSVM.csv', sep=';')
        else:
          df = pd.read_csv(path_results_test + dataset + '_' + best_modeling[met + '_3'][dataset] + '_features_' + met + '_3_OCSVM.csv', sep=';')

        f1 =  max(df['macro avg_f1-score-mean'])
        std = df[df['macro avg_f1-score-mean'] == f1]['macro avg_f1-score-std'].iloc[0]
        dict_test[met][dataset] = f1
        dict_test[met+'-std'][dataset] = std

      else:
        df = pd.read_csv(path_results_test + dataset + '_' + best_modeling[method][dataset] + '_' + method + '.csv', sep=';')
        f1 =  max(df['macro avg_f1-score-mean'])
        std = df[df['macro avg_f1-score-mean'] == f1]['macro avg_f1-score-std'].iloc[0]
        dict_test[method][dataset] = f1
        dict_test[method+'-std'][dataset] = std

  return dict_test

In [ ]:
path_results_test = '/content/drive/MyDrive/USP/Doctorate/Research/Articles/OLGA: One cLass Graph Autoencoder/results_test/8_datasets_2,3emb/'

dict_test = results_dataframe(path_results_test, best_modeling)

results_low_dimensions = pd.DataFrame(dict_test)
results_low_dimensions

,gae,gae-std,deepwalk,deepwalk-std,node2vec,node2vec-std,OC-GCN,OC-GCN-std,OC-GAT,OC-GAT-std,OC-GraphSAGE,OC-GraphSAGE-std,OLGA,OLGA-std
fakenews,0.949957,0.009723,0.868397,0.017919,0.892317,0.014691,0.634677,0.014201,0.506146,0.012833,0.485557,0.021631,0.939575,0.016301
terrorism,0.624205,0.011197,0.978288,0.003754,0.670550,0.014308,0.823612,0.011592,0.744323,0.012199,0.640248,0.010428,0.920675,0.006918
relevant_reviews,0.741780,0.009425,0.613849,0.016576,0.707351,0.007692,0.546220,0.006944,0.540844,0.004976,0.520643,0.007538,0.750486,0.007273
food,0.994821,0.003136,0.767947,0.011532,0.980416,0.003263,0.787033,0.016153,0.634716,0.015236,0.525636,0.011690,0.996875,0.001812
pneumonia,0.393775,0.000090,0.877690,0.008567,0.786612,0.009887,0.588494,0.009648,0.664225,0.012594,0.661093,0.006219,0.914024,0.010339
musk,0.478004,0.067842,0.725838,0.014594,0.525592,0.012752,0.504729,0.007923,0.437186,0.006571,0.476636,0.010721,0.785207,0.028812
TUANDROMD,0.862880,0.020328,0.688539,0.009436,0.823213,0.011770,0.756487,0.011312,0.598065,0.015555,0.853647,0.012066,0.974386,0.006333
strawberry,0.530724,0.017164,0.942366,0.013279,0.543665,0.021874,0.594289,0.023277,0.598097,0.015049,0.557322,0.013486,0.646702,0.100736


In [ ]:
results_low_dimensions.to_csv('low_dimension_icdm.csv')

In [ ]:
results_low_dimensions[['gae','deepwalk', 'node2vec', 'OC-GCN', 'OC-GAT', 'OC-GraphSAGE', 'OLGA']].to_csv('low_dimension_icdm-critical_diference.csv')

# High Dimension

In [ ]:
path_results_validation_128 = '/content/drive/MyDrive/USP/Doctorate/Research/Articles/OLGA: One cLass Graph Autoencoder/results/8_datasets_128_emb/'

dict_high = {'gae': {},
          'deepwalk': {},
          'node2vec': {},
          'gae': {},
          'OC-GCN-128': {},
          'OC-GAT-128': {},
          'OC-GraphSAGE-128': {},
          'OLGA': {}}

dict_validation_high = best_modeling_graph(path_results_validation_128, dict_high, True)
best_modeling_high = pd.DataFrame(dict_validation_high)
best_modeling_high

,gae,deepwalk,node2vec,OC-GCN-128,OC-GAT-128,OC-GraphSAGE-128,OLGA
fakenews,k=3,k=3,k=3,k=3,k=3,k=2,k=3
food,k=3,k=1,k=3,k=3,k=3,k=3,k=2
musk,k=2,k=1,k=3,k=2,k=3,k=2,k=1
pneumonia,k=3,k=1,k=3,k=2,k=3,k=3,k=3
relevant_reviews,k=3,k=1,k=3,k=3,k=3,k=2,k=3
strawberry,k=3,k=1,k=3,k=1,k=3,k=1,k=1
terrorism,k=1,k=1,k=3,k=3,k=3,k=1,k=3
TUANDROMD,k=1,k=3,k=3,k=1,k=1,k=1,k=3


In [ ]:
path_results_test_128 = '/content/drive/MyDrive/USP/Doctorate/Research/Articles/OLGA: One cLass Graph Autoencoder/results_test/8_datasets_128_emb/'


dict_test_high = {'gae': {},
               'gae-std': {},
              'deepwalk': {},
               'deepwalk-std': {},
              'node2vec': {},
               'node2vec-std': {},
              'OC-GCN-128': {},
                'OC-GCN-128-std': {},
              'OC-GAT-128': {},
               'OC-GAT-128-std': {},
              'OC-GraphSAGE-128': {},
               'OC-GraphSAGE-128-std': {},
              'OLGA': {},
              'OLGA-std': {}}

for dataset in best_modeling_high.index:
  for method in best_modeling_high.columns:
    if method == 'gae' or method == 'deepwalk' or method == 'node2vec':
      df = pd.read_csv(path_results_test_128 + dataset + '_' + best_modeling_high[method][dataset] + '_features_' + method + '_OCSVM.csv', sep=';')

    elif method == 'OC-GCN-128' or method == 'OC-GAT-128' or method == 'OC-GraphSAGE-128':
      df = pd.read_csv(path_results_test_128 + dataset + '_' + best_modeling_high[method][dataset] + '_' + method + '.csv', sep=';')

    else:
      df = pd.read_csv(path_results_test_128 + dataset + '_' + best_modeling_high[method][dataset] + '_' + method + '_128.csv', sep=';')

    f1 =  max(df['macro avg_f1-score-mean'])
    std = df[df['macro avg_f1-score-mean'] == f1]['macro avg_f1-score-std'].iloc[0]
    dict_test_high [method][dataset] = f1
    dict_test_high [method+'-std'][dataset] = std

results_high_dimensions = pd.DataFrame(dict_test_high)
results_high_dimensions

,gae,gae-std,deepwalk,deepwalk-std,node2vec,node2vec-std,OC-GCN-128,OC-GCN-128-std,OC-GAT-128,OC-GAT-128-std,OC-GraphSAGE-128,OC-GraphSAGE-128-std,OLGA,OLGA-std
fakenews,0.941765,0.011159,0.879052,0.015513,0.828689,0.022033,0.745864,0.045006,0.884135,0.017265,0.823924,0.015008,0.945143,0.012143
food,0.981661,0.004457,0.751726,0.011622,0.959867,0.007039,0.900643,0.009434,0.993546,0.002844,0.985145,0.003275,0.996513,0.002063
musk,0.622728,0.063007,0.774081,0.006354,0.639554,0.019159,0.665839,0.017693,0.620098,0.008470,0.602572,0.012547,0.713271,0.027748
pneumonia,0.638898,0.011674,0.877870,0.009766,0.644714,0.012037,0.505062,0.011993,0.751839,0.010907,0.655046,0.014344,0.917686,0.007503
relevant_reviews,0.727637,0.009354,0.597999,0.022578,0.644324,0.009109,0.592078,0.010036,0.690984,0.005536,0.622859,0.009686,0.771417,0.009078
strawberry,0.516230,0.020908,0.960554,0.006866,0.584041,0.030644,0.434533,0.093109,0.629162,0.019565,0.634891,0.024671,0.585172,0.040024
terrorism,0.727094,0.037994,0.981057,0.002908,0.799827,0.014622,0.844735,0.020637,0.899827,0.007451,0.785156,0.020005,0.895714,0.008899
TUANDROMD,0.904255,0.010834,0.688962,0.007375,0.904571,0.007409,0.898359,0.008494,0.978098,0.004160,0.973687,0.004032,0.969103,0.006194


In [ ]:
results_high_dimensions[['gae','deepwalk', 'node2vec', 'OC-GCN-128', 'OC-GAT-128', 'OC-GraphSAGE-128', 'OLGA']].to_csv('high_dimension_icdm-critical_diference.csv')

In [ ]:
results_high_dimensions.to_csv('high_dimension_icdm.csv')